In [56]:
# Import data packages
import pandas as pd
from pathlib import Path
import warnings

# Import database packages
from sqlalchemy import create_engine, select
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session, declarative_base



In [57]:
#Global configs
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [58]:
#Open data file into csv
file_path = Path("./data/zillow_listed_prices_052924.csv")

housing_data = pd.read_csv(file_path)
housing_data.head()

,address,addressCity,addressState,addressStreet,addressZipcode,area,availabilityDate,baths,beds,brokerName,countryCurrency,hasAdditionalAttributions,id,isFeaturedListing,isHomeRec,isSaved,isUndisclosedAddress,isUserClaimingOwner,latLong/latitude,latLong/longitude,list,openHouseDescription,openHouseStartDate,price,shouldShowZestimateAsPrice,statusText,statusType,unformattedPrice,variableData/text,variableData/type,zpid
0,"17620 N 17th Pl UNIT 34, Phoenix, AZ 85022",Phoenix,AZ,17620 N 17th Pl UNIT 34,85022,1207,NaN,3,2,"Opendoor Brokerage, LLC",$,True,8000871,False,False,False,False,False,33.646866,-112.045580,True,Open House - 8:00 AM - 7:30 PM,2024-05-30T08:00:00,"$317,000",False,Townhouse for sale,FOR_SALE,317000,Open: Thu. 8am-7:30pm,OPEN_HOUSE,8000871
1,"11061 N 111th Way, Scottsdale, AZ 85259",Scottsdale,AZ,11061 N 111th Way,85259,1471,NaN,2,3,HomeSmart,$,True,8050748,False,False,False,False,False,33.587170,-111.840965,True,Open House - 0:00 - 2:30 PM,2024-06-01T12:00:00,"$579,000",False,House for sale,FOR_SALE,579000,Open: Sat. 12-2:30pm,OPEN_HOUSE,8050748
2,"7751 E Glenrosa Ave APT C5, Scottsdale, AZ 85251",Scottsdale,AZ,7751 E Glenrosa Ave APT C5,85251,957,NaN,1,2,eXp Realty,$,True,7846319,False,False,False,False,False,33.498177,-111.914280,True,Open House - 10:00 AM - 1:00 PM,2024-06-01T10:00:00,"$370,000",False,Apartment for sale,FOR_SALE,370000,Open: Sat. 10am-1pm,OPEN_HOUSE,7846319
3,"7002 S 42nd Way, Phoenix, AZ 85042",Phoenix,AZ,7002 S 42nd Way,85042,939,NaN,2,2,Realty ONE Group,$,True,7552172,False,False,False,False,False,33.383450,-111.990260,True,Open House - 10:00 AM - 1:00 PM,2024-06-01T10:00:00,"$325,000",False,Townhouse for sale,FOR_SALE,325000,Open: Sat. 10am-1pm,OPEN_HOUSE,7552172
4,"8356 E San Ramon Dr, Scottsdale, AZ 85258",Scottsdale,AZ,8356 E San Ramon Dr,85258,2618,NaN,3,4,HomeSmart,$,True,7867033,False,False,False,False,False,33.569626,-111.900050,True,Open House - 0:00 - 4:00 PM,2024-06-01T12:00:00,"$1,200,000",False,House for sale,FOR_SALE,1200000,Open: Sat. 12-4pm,OPEN_HOUSE,7867033


In [59]:
housing_data.columns

Index(['address', 'addressCity', 'addressState', 'addressStreet',
       'addressZipcode', 'area', 'availabilityDate', 'baths', 'beds',
       'brokerName', 'countryCurrency', 'hasAdditionalAttributions', 'id',
       'isFeaturedListing', 'isHomeRec', 'isSaved', 'isUndisclosedAddress',
       'isUserClaimingOwner', 'latLong/latitude', 'latLong/longitude', 'list',
       'openHouseDescription', 'openHouseStartDate', 'price',
       'shouldShowZestimateAsPrice', 'statusText', 'statusType',
       'unformattedPrice', 'variableData/text', 'variableData/type', 'zpid'],
      dtype='object')

In [60]:
corr_data = housing_data[['area', 'baths', 'beds', 'latLong/latitude', 'latLong/longitude']].corrwith(housing_data['unformattedPrice'])
corr_data

area                 0.868648
baths                0.733971
beds                 0.480398
latLong/latitude     0.196374
latLong/longitude    0.356212
dtype: float64

In [61]:
housing_data.columns

Index(['address', 'addressCity', 'addressState', 'addressStreet',
       'addressZipcode', 'area', 'availabilityDate', 'baths', 'beds',
       'brokerName', 'countryCurrency', 'hasAdditionalAttributions', 'id',
       'isFeaturedListing', 'isHomeRec', 'isSaved', 'isUndisclosedAddress',
       'isUserClaimingOwner', 'latLong/latitude', 'latLong/longitude', 'list',
       'openHouseDescription', 'openHouseStartDate', 'price',
       'shouldShowZestimateAsPrice', 'statusText', 'statusType',
       'unformattedPrice', 'variableData/text', 'variableData/type', 'zpid'],
      dtype='object')

In [62]:
# Drop unnecessary columns
model_df = housing_data[['id', 'area', 'baths', 'beds', 'latLong/latitude', 'latLong/longitude', 'unformattedPrice']]
model_df.head()

,id,area,baths,beds,latLong/latitude,latLong/longitude,unformattedPrice
0,8000871,1207,3,2,33.646866,-112.045580,317000
1,8050748,1471,2,3,33.587170,-111.840965,579000
2,7846319,957,1,2,33.498177,-111.914280,370000
3,7552172,939,2,2,33.383450,-111.990260,325000
4,7867033,2618,3,4,33.569626,-111.900050,1200000


In [63]:
# Create df for reference data
ref_df = housing_data[['id', 'price', 'addressCity', 'addressZipcode', 'area', 'baths',
        'beds', 'latLong/latitude', 'latLong/longitude',  'statusText']]
ref_df.head()

,id,price,addressCity,addressZipcode,area,baths,beds,latLong/latitude,latLong/longitude,statusText
0,8000871,"$317,000",Phoenix,85022,1207,3,2,33.646866,-112.045580,Townhouse for sale
1,8050748,"$579,000",Scottsdale,85259,1471,2,3,33.587170,-111.840965,House for sale
2,7846319,"$370,000",Scottsdale,85251,957,1,2,33.498177,-111.914280,Apartment for sale
3,7552172,"$325,000",Phoenix,85042,939,2,2,33.383450,-111.990260,Townhouse for sale
4,7867033,"$1,200,000",Scottsdale,85258,2618,3,4,33.569626,-111.900050,House for sale


In [64]:
# Start model data cleaning

#rename columns
model_df = model_df.rename(columns={'id': 'id', 'area': 'sqft', 'latLong/latitude': 'latitude', 
                                    'latLong/longitude': 'longitude', 'unformattedPrice': 'sale_price'})

#Confirm final data
model_df.head()

,id,sqft,baths,beds,latitude,longitude,sale_price
0,8000871,1207,3,2,33.646866,-112.045580,317000
1,8050748,1471,2,3,33.587170,-111.840965,579000
2,7846319,957,1,2,33.498177,-111.914280,370000
3,7552172,939,2,2,33.383450,-111.990260,325000
4,7867033,2618,3,4,33.569626,-111.900050,1200000


In [65]:
#Start reference data cleansing
ref_df = ref_df.rename(columns={'addressCity': 'city', 'addressZipcode': 'zipcode', 'area': 'sqft', 
                        'latLong/latitude': 'latitude', 'latLong/longitude': 'longitude',  'statusText': 'type'})

#cleanup house type
ref_df['type'] = ref_df.apply(lambda row: row['type'].split(" ")[0], axis=1)

ref_df.head()

,id,price,city,zipcode,sqft,baths,beds,latitude,longitude,type
0,8000871,"$317,000",Phoenix,85022,1207,3,2,33.646866,-112.045580,Townhouse
1,8050748,"$579,000",Scottsdale,85259,1471,2,3,33.587170,-111.840965,House
2,7846319,"$370,000",Scottsdale,85251,957,1,2,33.498177,-111.914280,Apartment
3,7552172,"$325,000",Phoenix,85042,939,2,2,33.383450,-111.990260,Townhouse
4,7867033,"$1,200,000",Scottsdale,85258,2618,3,4,33.569626,-111.900050,House


In [66]:
#Get model columns for database
model_df.columns

Index(['id', 'sqft', 'baths', 'beds', 'latitude', 'longitude', 'sale_price'], dtype='object')

In [67]:
#Get reference columns for database
ref_df.columns

Index(['id', 'price', 'city', 'zipcode', 'sqft', 'baths', 'beds', 'latitude',
       'longitude', 'type'],
      dtype='object')

In [68]:
# Begin data classes and storage
Base = declarative_base()

# Create a connection to a SQLite database
engine = create_engine(f"sqlite:///housing_model.db")

In [69]:
#Define model and reference tables

class ModelData(Base):
    __tablename__ = 'model_data'
    id = Column(Integer, primary_key=True)
    beds = Column(Integer)
    baths = Column(Integer)
    sqft = Column(Integer)
    latitude = Column(Integer)
    longitude = Column(Integer)
    price = Column(Integer)

class RefData(Base):
    __tablename__ = 'reference_data'
    id = Column(Integer, primary_key=True)
    price = Column(String)
    city = Column(String)
    zipcode = Column(String)
    sqft = Column(Integer)
    baths = Column(Integer)
    beds = Column(Integer)
    latitude = Column(Float)
    longitude = Column(Float)
    type = Column(String)

In [75]:
# Create the travel_destinations table within the database
Base.metadata.create_all(engine)

In [76]:
# Loop thru dataframes and write to tables

with Session(engine) as session:
    #Loop thru model data
    for index, row in model_df.iterrows():
        model = ModelData()
        model.id = row['id']
        model.beds = row['beds']
        model.baths = row['baths']
        model.sqft = row['sqft']
        model.latitude = row['latitude']
        model.longitude = row['longitude']
        model.price = row['sale_price']

        session.add(model)

    #commit after adding all rows for each model
    session.commit()

    #Loop thru reference data
    for index, row in ref_df.iterrows():
        ref = RefData()
        ref.id = row['id']
        ref.price = row['price']
        ref.city = row['city']
        ref.zipcode = row['zipcode']
        ref.sqft = row['sqft']
        ref.baths = row['baths']
        ref.beds = row['beds']
        ref.latitude = row['latitude']
        ref.longitude = row['longitude']
        ref.type = row['type']

        session.add(ref)

    #commit after adding all rows for each model
    session.commit()
    

In [77]:
#Validate data in database
#Model data
stmt = select(ModelData)
results = session.execute(stmt).mappings().all()
for row in results:
    print(f'ID: {row.ModelData.id} - Price: {row.ModelData.price}')

ID: 7467082 - Price: 225000
ID: 7468441 - Price: 399000
ID: 7470876 - Price: 365000
ID: 7480371 - Price: 362000
ID: 7483437 - Price: 346000
ID: 7499343 - Price: 375000
ID: 7499667 - Price: 327000
ID: 7514293 - Price: 369000
ID: 7514792 - Price: 375000
ID: 7519342 - Price: 540000
ID: 7520363 - Price: 480000
ID: 7520505 - Price: 1750000
ID: 7520682 - Price: 774900
ID: 7520958 - Price: 899900
ID: 7521188 - Price: 550000
ID: 7526319 - Price: 815000
ID: 7530024 - Price: 578499
ID: 7532369 - Price: 750000
ID: 7533535 - Price: 450000
ID: 7538431 - Price: 619000
ID: 7541005 - Price: 368000
ID: 7552172 - Price: 325000
ID: 7561940 - Price: 530000
ID: 7564220 - Price: 1850000
ID: 7564928 - Price: 844999
ID: 7565096 - Price: 2499999
ID: 7565633 - Price: 750000
ID: 7567545 - Price: 650000
ID: 7567632 - Price: 640000
ID: 7569304 - Price: 859000
ID: 7570063 - Price: 595000
ID: 7571951 - Price: 640000
ID: 7572354 - Price: 2300000
ID: 7573099 - Price: 519900
ID: 7574293 - Price: 459000
ID: 7574397 - Pr

In [78]:
#Reference data
stmt = select(RefData)
results = session.execute(stmt).mappings().all()
for row in results:
    print(f'ID: {row.RefData.id} - Price: {row.RefData.price}')

ID: 7467082 - Price: $225K
ID: 7468441 - Price: $399,000
ID: 7470876 - Price: $365,000
ID: 7480371 - Price: $362,000
ID: 7483437 - Price: $346,000
ID: 7499343 - Price: $375,000
ID: 7499667 - Price: $327,000
ID: 7514293 - Price: $369,000
ID: 7514792 - Price: $375,000
ID: 7519342 - Price: $540,000
ID: 7520363 - Price: $480,000
ID: 7520505 - Price: $1,750,000
ID: 7520682 - Price: $774,900
ID: 7520958 - Price: $899,900
ID: 7521188 - Price: $550,000
ID: 7526319 - Price: $815,000
ID: 7530024 - Price: $578,499
ID: 7532369 - Price: $750,000
ID: 7533535 - Price: $450,000
ID: 7538431 - Price: $619,000
ID: 7541005 - Price: $368,000
ID: 7552172 - Price: $325,000
ID: 7561940 - Price: $530,000
ID: 7564220 - Price: $1,850,000
ID: 7564928 - Price: $844,999
ID: 7565096 - Price: $2,499,999
ID: 7565633 - Price: $750,000
ID: 7567545 - Price: $650,000
ID: 7567632 - Price: $640,000
ID: 7569304 - Price: $859,000
ID: 7570063 - Price: $595,000
ID: 7571951 - Price: $640,000
ID: 7572354 - Price: $2,300,000
ID: 7

In [79]:
session.close()